<a href="https://colab.research.google.com/github/kanru-wang/naacl_nlp_transfer_learning_tutorial/blob/master/Document_level_classification_using_fast_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook accompanying NAACL 2019 tutorial on "Transfer Learning in Natural Language Processing".

The tutorial will be given on June 2 at NAACL 2019 in Minneapolis, MN, USA by [Sebastian Ruder](http://ruder.io/), [Matthew Peters](https://www.linkedin.com/in/petersmatthew), [Swabha Swayamdipta](http://www.cs.cmu.edu/~sswayamd/index.html) and [Thomas Wolf](http://thomwolf.io/).

You can check the [webpage](https://naacl2019.org/program/tutorials/) of NAACL tutorials for more information.

Further material: [slides](http://tiny.cc/NAACLTransfer) and [code](http://tiny.cc/NAACLTransferCode).

# Document level classification

In this notebook, we recreated the [text classification example](https://github.com/fastai/fastai/blob/master/examples/text.ipynb) by Sylvain Gugger from fast.ai which show how to do transfer learning using the [fast.ai](https://github.com/fastai/fastai) library.

This example consists in creating a language model and then transfering it to a classifier.

In [0]:
from fastai.text import *   # Quick access to NLP functionality

## Get a document classification dataset: IMDB

Download the IMDB dataset and inspect itcreate

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)
print("Path:", path)
df = pd.read_csv(path/'texts.csv')
df.head()

Path: /root/.fastai/data/imdb_sample


,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Create a DataBunch for each of the language model and the classifier:

In [0]:
data_lm  = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv',
                       vocab=data_lm.train_ds.vocab, bs=42)

We'll fine-tune the language model. fast.ai has a pre-trained English model available that we can download, we just have to specify it like this:

In [0]:
moms = (0.8,0.7)
learn = language_model_learner(data_lm, AWD_LSTM)
learn.unfreeze()
learn.fit_one_cycle(4, slice(1e-2), moms=moms)
learn.save_encoder('enc')

epoch,train_loss,valid_loss,accuracy,time
0,4.723435,3.968737,0.283498,00:15
1,4.416326,3.874095,0.286878,00:15
2,4.148463,3.836543,0.290434,00:16
3,3.951989,3.828021,0.291311,00:16


Fine tune it to create a classifier:


In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM)
learn.load_encoder('enc')
learn.fit_one_cycle(4, moms=moms)

epoch,train_loss,valid_loss,accuracy,time
0,0.663383,0.682115,0.572139,00:10
1,0.623683,0.609520,0.651741,00:10
2,0.597989,0.582999,0.666667,00:10
3,0.580533,0.555404,0.666667,00:09


In [0]:
learn.unfreeze()
learn.fit_one_cycle(8, slice(1e-5,1e-3), moms=moms)

epoch,train_loss,valid_loss,accuracy,time
0,0.555569,0.557091,0.681592,00:20
1,0.566048,0.541689,0.721393,00:21
2,0.554564,0.543157,0.736318,00:20
3,0.556879,0.526971,0.756219,00:20
4,0.552898,0.522964,0.751244,00:19
5,0.541698,0.514611,0.756219,00:19
6,0.535575,0.514330,0.756219,00:19
7,0.529567,0.515582,0.746269,00:19
